<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/roberta_base_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [ ]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 31.1 MB/s 
     |████████████████████████████████| 432 kB 65.6 MB/s 
     |████████████████████████████████| 163 kB 72.4 MB/s 
     |████████████████████████████████| 7.6 MB 66.3 MB/s 
     |████████████████████████████████| 115 kB 76.4 MB/s 
     |████████████████████████████████| 212 kB 75.8 MB/s 
     |████████████████████████████████| 127 kB 75.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [ ]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  6821k      0 --:--:-- --:--:-- --:--:-- 6794k


In [ ]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForMaskedLM.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [ ]:
!shuf -n1000 kant.txt > kant_teste.txt

In [ ]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-362ca44bf038f2d2/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (592 > 512). Running this sequence through the model will result in indexing errors


...

Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors


#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


exemplo de entrada

In [ ]:
tokenized_datasets["train"][4]

{'input_ids': [0, 9167, 9942, 5563, 11326, 493, 263, 42262, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 32


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],    
)

In [ ]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 21430
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13400


Step,Training Loss
500,2.846200
1000,2.509500
1500,2.364800
2000,2.242700
2500,2.190600
3000,2.054100
3500,2.039100
4000,1.995700
4500,1.932700
5000,1.884700


Saving model checkpoint to ./KantaiBERT/checkpoint-10000
Configuration saved in ./KantaiBERT/checkpoint-10000/config.json
Model weights saved in ./KantaiBERT/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 36min 21s, sys: 3min 9s, total: 39min 31s
Wall time: 39min 36s


TrainOutput(global_step=13400, training_loss=1.8651716454349347, metrics={'train_runtime': 2376.1485, 'train_samples_per_second': 90.188, 'train_steps_per_second': 5.639, 'total_flos': 3526111799020800.0, 'train_loss': 1.8651716454349347, 'epoch': 10.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 2669
  Batch size = 8


Perplexity: 4.31


In [ ]:
eval_results

{'eval_loss': 1.4610384702682495,
 'eval_runtime': 8.446,
 'eval_samples_per_second': 316.008,
 'eval_steps_per_second': 39.545,
 'epoch': 10.0}

In [ ]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [ ]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout

In [ ]:
fill_mask("O rapaz olhou para o <mask>")

[{'score': 0.2596186101436615,
  'token': 13305,
  'token_str': ' piano',
  'sequence': 'O rapaz olhou para o piano'},
 {'score': 0.12805843353271484,
  'token': 4401,
  'token_str': ' mar',
  'sequence': 'O rapaz olhou para o mar'},
 {'score': 0.11640520393848419,
  'token': 30449,
  'token_str': ' altar',
  'sequence': 'O rapaz olhou para o altar'},
 {'score': 0.055015403777360916,
  'token': 4709,
  'token_str': ' ar',
  'sequence': 'O rapaz olhou para o ar'},
 {'score': 0.05430812016129494,
  'token': 26059,
  'token_str': ' café',
  'sequence': 'O rapaz olhou para o café'}]

In [ ]:
fill_mask("A moça olhou para o <mask>")

[{'score': 0.23862844705581665,
  'token': 13305,
  'token_str': ' piano',
  'sequence': 'A moça olhou para o piano'},
 {'score': 0.163367360830307,
  'token': 30449,
  'token_str': ' altar',
  'sequence': 'A moça olhou para o altar'},
 {'score': 0.1434350311756134,
  'token': 4401,
  'token_str': ' mar',
  'sequence': 'A moça olhou para o mar'},
 {'score': 0.08001280575990677,
  'token': 26059,
  'token_str': ' café',
  'sequence': 'A moça olhou para o café'},
 {'score': 0.061884086579084396,
  'token': 4709,
  'token_str': ' ar',
  'sequence': 'A moça olhou para o ar'}]

In [ ]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.12300211191177368,
  'token': 18236,
  'token_str': ' pasta',
  'sequence': 'Comprou uma pasta na loja. '},
 {'score': 0.07788186520338058,
  'token': 20889,
  'token_str': ' tempo',
  'sequence': 'Comprou uma tempo na loja. '},
 {'score': 0.07557501643896103,
  'token': 4709,
  'token_str': ' ar',
  'sequence': 'Comprou uma ar na loja. '},
 {'score': 0.06064959987998009,
  'token': 38996,
  'token_str': ' manga',
  'sequence': 'Comprou uma manga na loja. '},
 {'score': 0.06034652888774872,
  'token': 26059,
  'token_str': ' café',
  'sequence': 'Comprou uma café na loja. '}]

In [ ]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.40817591547966003,
  'token': 1632,
  'token_str': ' natural',
  'sequence': 'A mulher não é natural. '},
 {'score': 0.09705601632595062,
  'token': 44010,
  'token_str': ' bom',
  'sequence': 'A mulher não é bom. '},
 {'score': 0.07935484498739243,
  'token': 9753,
  'token_str': ' grave',
  'sequence': 'A mulher não é grave. '},
 {'score': 0.05318139120936394,
  'token': 1192,
  'token_str': ' que',
  'sequence': 'A mulher não é que. '},
 {'score': 0.04522813856601715,
  'token': 2950,
  'token_str': ' familiar',
  'sequence': 'A mulher não é familiar. '}]

In [ ]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.19068579375743866,
  'token': 44010,
  'token_str': ' bom',
  'sequence': 'O homem não é bom. '},
 {'score': 0.16067355871200562,
  'token': 9753,
  'token_str': ' grave',
  'sequence': 'O homem não é grave. '},
 {'score': 0.09638561308383942,
  'token': 1632,
  'token_str': ' natural',
  'sequence': 'O homem não é natural. '},
 {'score': 0.06944520771503448,
  'token': 1192,
  'token_str': ' que',
  'sequence': 'O homem não é que. '},
 {'score': 0.047215916216373444,
  'token': 8196,
  'token_str': ' mal',
  'sequence': 'O homem não é mal. '}]

In [ ]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.10405716300010681,
  'token': 45345,
  'token_str': ' Deus',
  'sequence': 'Ele é um bom Deus. '},
 {'score': 0.0997682511806488,
  'token': 3477,
  'token_str': ' animal',
  'sequence': 'Ele é um bom animal. '},
 {'score': 0.04183599352836609,
  'token': 44010,
  'token_str': ' bom',
  'sequence': 'Ele é um bom bom. '},
 {'score': 0.04088491201400757,
  'token': 538,
  'token_str': ' major',
  'sequence': 'Ele é um bom major. '},
 {'score': 0.0318363755941391,
  'token': 22677,
  'token_str': ' rumor',
  'sequence': 'Ele é um bom rumor. '}]

In [ ]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.09774405509233475,
  'token': 35277,
  'token_str': ' arte',
  'sequence': 'Ela é uma boa arte. '},
 {'score': 0.05576702952384949,
  'token': 652,
  'token_str': ' face',
  'sequence': 'Ela é uma boa face. '},
 {'score': 0.03390379250049591,
  'token': 4149,
  'token_str': ' drama',
  'sequence': 'Ela é uma boa drama. '},
 {'score': 0.03377309441566467,
  'token': 18236,
  'token_str': ' pasta',
  'sequence': 'Ela é uma boa pasta. '},
 {'score': 0.032380763441324234,
  'token': 38996,
  'token_str': ' manga',
  'sequence': 'Ela é uma boa manga. '}]

In [ ]:
fill_mask("Faz de conta que ainda é <mask>. ")

[{'score': 0.47132498025894165,
  'token': 44010,
  'token_str': ' bom',
  'sequence': 'Faz de conta que ainda é bom. '},
 {'score': 0.18704472482204437,
  'token': 1632,
  'token_str': ' natural',
  'sequence': 'Faz de conta que ainda é natural. '},
 {'score': 0.05797050520777702,
  'token': 1192,
  'token_str': ' que',
  'sequence': 'Faz de conta que ainda é que. '},
 {'score': 0.046140432357788086,
  'token': 9753,
  'token_str': ' grave',
  'sequence': 'Faz de conta que ainda é grave. '},
 {'score': 0.02763802744448185,
  'token': 8196,
  'token_str': ' mal',
  'sequence': 'Faz de conta que ainda é mal. '}]

In [ ]:
fill_mask("Depois da tempestade vem a <mask>. ")

[{'score': 0.1570359468460083,
  'token': 23684,
  'token_str': ' mim',
  'sequence': 'Depois da tempestade vem a mim. '},
 {'score': 0.11335571110248566,
  'token': 11608,
  'token_str': ' Europa',
  'sequence': 'Depois da tempestade vem a Europa. '},
 {'score': 0.07653415948152542,
  'token': 3391,
  'token_str': ' si',
  'sequence': 'Depois da tempestade vem a si. '},
 {'score': 0.0719151571393013,
  'token': 8865,
  'token_str': 'inda',
  'sequence': 'Depois da tempestade vem ainda. '},
 {'score': 0.05291016772389412,
  'token': 4342,
  'token_str': ' ver',
  'sequence': 'Depois da tempestade vem a ver. '}]

In [ ]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

[{'score': 0.46646711230278015,
  'token': 7935,
  'token_str': ' é',
  'sequence': 'Mais vale um pássaro na mão do que é voando. '},
 {'score': 0.11656363308429718,
  'token': 6,
  'token_str': ',',
  'sequence': 'Mais vale um pássaro na mão do que, voando. '},
 {'score': 0.0781007781624794,
  'token': 11030,
  'token_str': ' tem',
  'sequence': 'Mais vale um pássaro na mão do que tem voando. '},
 {'score': 0.03973055258393288,
  'token': 162,
  'token_str': ' me',
  'sequence': 'Mais vale um pássaro na mão do que me voando. '},
 {'score': 0.03796108439564705,
  'token': 1021,
  'token_str': ' o',
  'sequence': 'Mais vale um pássaro na mão do que o voando. '}]

In [ ]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

[{'score': 0.5427026152610779,
  'token': 20889,
  'token_str': ' tempo',
  'sequence': 'Não tinha tempo, mas almoçou mesmo assim. '},
 {'score': 0.09713822603225708,
  'token': 8196,
  'token_str': ' mal',
  'sequence': 'Não tinha mal, mas almoçou mesmo assim. '},
 {'score': 0.03522020950913429,
  'token': 4402,
  'token_str': ' favor',
  'sequence': 'Não tinha favor, mas almoçou mesmo assim. '},
 {'score': 0.022530443966388702,
  'token': 26059,
  'token_str': ' café',
  'sequence': 'Não tinha café, mas almoçou mesmo assim. '},
 {'score': 0.021098807454109192,
  'token': 6655,
  'token_str': ' logo',
  'sequence': 'Não tinha logo, mas almoçou mesmo assim. '}]

In [ ]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

[{'score': 0.12842363119125366,
  'token': 20889,
  'token_str': ' tempo',
  'sequence': "Bebeu um copo d'água, pois tinha tempo. "},
 {'score': 0.11853910982608795,
  'token': 16743,
  'token_str': ' ele',
  'sequence': "Bebeu um copo d'água, pois tinha ele. "},
 {'score': 0.06533609330654144,
  'token': 7252,
  'token_str': ' um',
  'sequence': "Bebeu um copo d'água, pois tinha um. "},
 {'score': 0.055333275347948074,
  'token': 8196,
  'token_str': ' mal',
  'sequence': "Bebeu um copo d'água, pois tinha mal. "},
 {'score': 0.04606688767671585,
  'token': 6655,
  'token_str': ' logo',
  'sequence': "Bebeu um copo d'água, pois tinha logo. "}]

In [ ]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

[{'score': 0.09936946630477905,
  'token': 22677,
  'token_str': ' rumor',
  'sequence': 'Sem saber, ele descobriu um rumor. '},
 {'score': 0.09179303050041199,
  'token': 23681,
  'token_str': ' tom',
  'sequence': 'Sem saber, ele descobriu um tom. '},
 {'score': 0.0705951377749443,
  'token': 4402,
  'token_str': ' favor',
  'sequence': 'Sem saber, ele descobriu um favor. '},
 {'score': 0.06321502476930618,
  'token': 26059,
  'token_str': ' café',
  'sequence': 'Sem saber, ele descobriu um café. '},
 {'score': 0.05145813897252083,
  'token': 4709,
  'token_str': ' ar',
  'sequence': 'Sem saber, ele descobriu um ar. '}]

In [ ]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

[{'score': 0.1511716991662979,
  'token': 6655,
  'token_str': ' logo',
  'sequence': 'Pedro fez fortuna vendendo logo. '},
 {'score': 0.13522399961948395,
  'token': 16743,
  'token_str': ' ele',
  'sequence': 'Pedro fez fortuna vendendo ele. '},
 {'score': 0.10773566365242004,
  'token': 1192,
  'token_str': ' que',
  'sequence': 'Pedro fez fortuna vendendo que. '},
 {'score': 0.0984577089548111,
  'token': 8196,
  'token_str': ' mal',
  'sequence': 'Pedro fez fortuna vendendo mal. '},
 {'score': 0.02789783850312233,
  'token': 20889,
  'token_str': ' tempo',
  'sequence': 'Pedro fez fortuna vendendo tempo. '}]

In [ ]:
fill_mask("Sem medo de ser <mask>. ")

[{'score': 0.16314978897571564,
  'token': 44010,
  'token_str': ' bom',
  'sequence': 'Sem medo de ser bom. '},
 {'score': 0.0481211319565773,
  'token': 7252,
  'token_str': ' um',
  'sequence': 'Sem medo de ser um. '},
 {'score': 0.036062344908714294,
  'token': 2950,
  'token_str': ' familiar',
  'sequence': 'Sem medo de ser familiar. '},
 {'score': 0.030418584123253822,
  'token': 45345,
  'token_str': ' Deus',
  'sequence': 'Sem medo de ser Deus. '},
 {'score': 0.0287080779671669,
  'token': 1192,
  'token_str': ' que',
  'sequence': 'Sem medo de ser que. '}]